<a href="https://colab.research.google.com/github/PratiteeMalakar/hello-world/blob/main/project6_industry_trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

# Example function to scrape competitor website
def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Extract text data
    text_data = ' '.join([p.text for p in soup.find_all('p')])
    return text_data

# Example function to get data from Twitter using Tweepy
# import tweepy

# def get_twitter_data(api_key, api_secret_key, access_token, access_token_secret, username):
#     auth = tweepy.OAuthHandler(api_key, api_secret_key)
#     auth.set_access_token(access_token, access_token_secret)
#     api = tweepy.API(auth)
#     tweets = api.user_timeline(screen_name=username, count=200, tweet_mode='extended')
#     tweet_texts = [tweet.full_text for tweet in tweets]
#     return ' '.join(tweet_texts)


In [ ]:
!pip install langchain-openai unstructured==0.7.12 pinecone-client openai tiktoken langchain clean-text langchain-pinecone langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import pinecone

In [ ]:
from pinecone import Pinecone
import os
from google.colab import userdata

os.environ["PINECONE_API_KEY"]=userdata.get("PINECONE")

api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key = api_key)

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud , region=region)

In [ ]:
import time
index_name = "index"

pc.create_index(index_name , dimension=1024, metric='cosine', spec=spec)



while not pc.describe_index(index_name).status['ready']:
  time.sleep(1)

In [ ]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

DATA PROCESSING

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
import mistralai
import numpy as np

# Set up Mistral.ai API key
#mistralai.api_key = 'YOUR_MISTRALAI_API_KEY'
os.environ["MISTRAL_API_KEY"]=userdata.get("MISTRAL")

mistral_api_key = os.getenv("MISTRAL_API_KEY")
from mistralai.client import MistralClient

client = MistralClient(api_key=mistral_api_key)


# Function to generate embeddings using Mistral.ai
def generate_embeddings(text):
    response = client.embeddings(
        input=text,
        model="mistral-embed"  # You can choose an appropriate model for your use case
    )
    return response.data[0].embedding

In [ ]:
pip install mistralai

In [ ]:
from mistralai.models.chat_completion import ChatMessage

# Function to add data to Pinecone
# Function to add data to Pinecone
def add_data_to_index(text_data, doc_id):
    embeddings = generate_embeddings(text_data)
    index.upsert([
        {
            'id': f'doc_{doc_id}',
            'values': embeddings,
            'metadata': {'text': text_data}
        }
    ])
    return embeddings


# Function to retrieve similar documents
def retrieve_similar_docs(query, k=5):
    query_embedding = generate_embeddings(query)
    response = index.query(vector=query_embedding, top_k=k)
    return response['matches']

# Function to generate responses using Mistral.ai
# Function to generate responses using Mistral.ai
# Function to generate responses using Mistral.ai
# Function to generate responses using Mistral.ai
def generate_response(query):
    response = index.query(vector=generate_embeddings(query), top_k=5, include_metadata=True)
    similar_docs = response['matches']

    # Check if any similar documents were found
    if similar_docs:
        # Handle potential missing metadata
        context = ' '.join([doc.metadata.get('text', '') if doc.metadata else '' for doc in similar_docs])  # Handle cases where doc.metadata might be None
        input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
        response = client.chat(
            model="open-mistral-7b",  # Replace with the actual free model name if available
            messages = [ChatMessage(role="user" , content = input_text)]
        )
        return response.choices[0].message.content
    else:
        return "No similar documents found. Please try a different query." # Return a message if no similar documents are foundurn a message if no similar documents are found

In [ ]:

# ... (rest of your code)

# Scrape competitor website
website_text = scrape_website('http://adasci.org')

# Get Twitter data
# ... (your Twitter data fetching code)

# Add data to index and print the embeddings
embeddings = add_data_to_index(website_text, doc_id=1)
print("Embeddings generated and added to index:", embeddings)

# Query the system and print similar documents
query = "What are the recent technological advancements in the industry?"
response = index.query(vector=generate_embeddings(query), top_k=5, include_metadata=True)
#print("Similar documents found:", response['matches'])

response_text = generate_response(query)
print(response_text)


Embeddings generated and added to index: [-0.0017938613891601562, 0.0330810546875, 0.03521728515625, 0.0166778564453125, 0.0089263916015625, 0.02374267578125, 0.054473876953125, -0.00229644775390625, -0.01241302490234375, 0.0067138671875, -0.01593017578125, 0.0660400390625, -0.02838134765625, -0.01230621337890625, -0.1055908203125, 0.048553466796875, 0.00829315185546875, 0.0276641845703125, 0.031585693359375, 0.0005869865417480469, -0.0298919677734375, -0.00994873046875, -0.041107177734375, 0.0157318115234375, -0.01248931884765625, -0.00885009765625, 0.005245208740234375, -0.06182861328125, -0.036773681640625, -0.001415252685546875, 0.036346435546875, -0.0283203125, -0.0301971435546875, 0.031341552734375, -0.021575927734375, -0.0186614990234375, -0.041290283203125, -0.038116455078125, 0.00101470947265625, 0.0030345916748046875, -0.00982666015625, -0.00748443603515625, 0.0211639404296875, -0.0146026611328125, 0.0005803108215332031, -0.0005664825439453125, -0.0005793571472167969, -0.0243